# Import & Configs

In [2]:
import sqlite3

import pandas as pd

In [40]:
DB_FILE = "../S30 ETL Assignment.db"
SOLUTION_CSV = "Solution.csv"

In [8]:
conn = sqlite3.connect(DB_FILE)
cur = conn.cursor()

(1, 10)

# Verify Tables

## Sales

In [10]:
cur.execute("select * from Sales;").fetchone()

(1, 10)

## Customers

In [13]:
cur.execute("select * from Customers;").fetchone()

(1, 38)

## Orders

In [12]:
cur.execute("select * from Orders;").fetchone()

(1, 1, 1, 2)

## Items

In [14]:
cur.execute("select * from Items;").fetchone()

(1, 'x')

# Load Into Pandas

## Sales

In [15]:
SALES_QUERY = """
select
    *
from Sales;
"""
sales = pd.read_sql(SALES_QUERY, con=conn)
display(sales.shape)

sales.head()

(500, 2)

,sales_id,customer_id
0,1,10
1,2,85
2,3,42
3,4,93
4,5,75


## Customers

In [16]:
CUSTOMERS_QUERY = """
select
    *
from Customers;
"""
customers = pd.read_sql(CUSTOMERS_QUERY, con=conn)
display(customers.shape)

customers.head()

(100, 2)

,customer_id,age
0,1,38
1,2,43
2,3,13
3,4,39
4,5,18


## Orders

In [17]:
ORDERS_QUERY = """
select
    *
from Orders;
"""
orders = pd.read_sql(ORDERS_QUERY, con=conn)
display(orders.shape)

orders.head()

(1500, 4)

,order_id,sales_id,item_id,quantity
0,1,1,1,2.0
1,2,1,2,NaN
2,3,1,3,1.0
3,4,2,1,1.0
4,5,2,2,1.0


## Items

In [18]:
ITEMS_QUERY = """
select
    *
from Items;
"""
items = pd.read_sql(ITEMS_QUERY, con=conn)
display(items.shape)

items.head()

(3, 2)

,item_id,item_name
0,1,x
1,2,y
2,3,z


## Solution

In [20]:
customers_subset = customers.loc[customers["age"].between(18, 35)].copy()
customers_subset.shape

(24, 2)

In [29]:
customers_sales = customers_subset.merge(sales, on="customer_id")
display(customers_sales.shape)

customers_sales.head()

(128, 3)

,customer_id,age,sales_id
0,5,18,48
1,5,18,171
2,5,18,183
3,5,18,405
4,7,30,29


In [30]:
customers_orders = customers_sales.merge(orders, on="sales_id")
display(customers_orders.shape)

customers_orders.head()

(384, 6)

,customer_id,age,sales_id,order_id,item_id,quantity
0,5,18,48,142,1,NaN
1,5,18,48,143,2,NaN
2,5,18,48,144,3,NaN
3,5,18,171,511,1,1.0
4,5,18,171,512,2,3.0


In [31]:
customers_items = customers_orders.merge(items, on="item_id")
display(customers_items.shape)

customers_items.head()

(384, 7)

,customer_id,age,sales_id,order_id,item_id,quantity,item_name
0,5,18,48,142,1,NaN,x
1,5,18,171,511,1,1.0,x
2,5,18,183,547,1,1.0,x
3,5,18,405,1213,1,1.0,x
4,7,30,29,85,1,NaN,x


In [32]:
customer_items_notnull = customers_items.loc[customers_items["quantity"].notna()]
display(customer_items_notnull.shape)

customer_items_notnull.head()

(238, 7)

,customer_id,age,sales_id,order_id,item_id,quantity,item_name
1,5,18,171,511,1,1.0,x
2,5,18,183,547,1,1.0,x
3,5,18,405,1213,1,1.0,x
5,7,30,36,106,1,2.0,x
6,7,30,53,157,1,1.0,x


In [44]:
total_items_per_customer = customer_items_notnull.groupby(["customer_id", "age", "item_name"]).sum()[["quantity"]].reset_index()
total_items_per_customer = total_items_per_customer.rename(columns={
    "customer_id": "Customer",
    "age": "Age",
    "item_name": "Item",
    "quantity": "Quantity",
})
total_items_per_customer["Quantity"] = total_items_per_customer["Quantity"].astype(int)
total_items_per_customer

,Customer,Age,Item,Quantity
0,5,18,x,3
1,5,18,y,3
2,5,18,z,4
3,7,30,x,14
4,7,30,y,19
...,...,...,...,...
65,97,23,y,2
66,97,23,z,3
67,100,30,x,1
68,100,30,y,1


In [46]:
total_items_per_customer.to_csv(SOLUTION_CSV, ";", index=False)
conn.close()